In [ ]:
class block(nn.Module):

    def __init__(self,in_channels=None,out_channels=None,stride=None):
        super(block,self).__init__()
        self.stride = stride 
        self.in_channels = in_channels 
        self.out_channels = out_channels
        self.nl = nn.ReLU()
        #self.s = nn.Identity()
        b1 = [nn.Conv2d(in_channels=self.in_channels, out_channels=self.out_channels,
                                                  kernel_size=(3,3), stride=self.stride, padding=(1,1)),nn.BatchNorm2d(num_features=self.out_channels)]
        self.x = nn.Sequential(*b1)
        b2 = [nn.Conv2d(in_channels=self.in_channels, out_channels=self.out_channels,
                                                  kernel_size=(1,1), stride=self.stride, padding=(0,0)),nn.BatchNorm2d(num_features=self.out_channels)]
        self.y = nn.Sequential(*b2)
        if self.stride == (1,1):
          self.z = nn.Identity()
        else :
          self.z =None

    def forward(self,input):
        if self.z == None :
            out = self.x(input) + self.y(input)
        else :
            out = self.x(input) + self.y(input) +self.z(input)
        out = self.nl(out)
        return out 


class REPVGG(nn.Module):

          def __init__(self,in_channels=3,num_classes=10,blocks=None,multipl=None):
            super(REPVGG,self).__init__()
            self.blocks= blocks
            self.multipl= multipl 
            self.in_channels = in_channels
            self.main_REPVGG = self.main_architecture()
            self.g = nn.AdaptiveAvgPool2d(output_size=1)
            self.linear = nn.Linear(int(512 * multipl[3]), num_classes)                               
          
          def main_architecture(self):
                    layers = []


                    #stage0

                    
                    multipl = self.multipl
                    out_channels = min(64,int(64*multipl[0])) 
                    in_channels = self.in_channels
                    blocks = self.blocks
                    layers += [block(in_channels = in_channels,out_channels=out_channels,stride = (2,2))]
                    in_channels = min(64, int(64*multipl[0]))




                    #stage1,stage2,stage3,stage4



                    for i in range(4):

                      out_channels = int(64*(2**i) * multipl[i])
                      layers += [block(in_channels = in_channels,out_channels=out_channels,stride = (2,2))]
                      in_channels = out_channels

                      for j in range(blocks[i]-1):
                        layers += [block(in_channels = in_channels,out_channels=out_channels,stride = (1,1))]
                      in_channels = out_channels 

                    return nn.Sequential(*layers)

          def forward(self,x):
            x = self.main_REPVGG(x)
            x = self.g(x)
            x = x.view(x.size(0), -1)
            x = self.linear(x)
            return x

In [ ]:
from torchsummary import summary

model_stats = summary(model, (3, 224, 224))
summary_str = str(model_stats)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 48, 112, 112]           1,344
       BatchNorm2d-2         [-1, 48, 112, 112]              96
            Conv2d-3         [-1, 48, 112, 112]             192
       BatchNorm2d-4         [-1, 48, 112, 112]              96
              ReLU-5         [-1, 48, 112, 112]               0
             block-6         [-1, 48, 112, 112]               0
            Conv2d-7           [-1, 48, 56, 56]          20,784
       BatchNorm2d-8           [-1, 48, 56, 56]              96
            Conv2d-9           [-1, 48, 56, 56]           2,352
      BatchNorm2d-10           [-1, 48, 56, 56]              96
             ReLU-11           [-1, 48, 56, 56]               0
            block-12           [-1, 48, 56, 56]               0
           Conv2d-13           [-1, 48, 56, 56]          20,784
      BatchNorm2d-14           [-1, 48,